In [48]:
import pyodbc  

DataSource = "DSN=BaseSQL32"
ExportFile = "PyUnload"
ExportDataDir = "C:\\PyUnload2\\PyUnload\\"
Usuarios = ['PRG']
#Usuarios = ["Productos", "Gestores", "Usuarios", "Inventario", "Empresas"]
maxRows = 100

In [57]:
# Estrae la lista de TABLAS y VISTAS de un propietario especifico
 
def crearModeloDatos(DataSource, Usuarios):

    modeloDatos = {}
    creator_id = ""
    conexionDB = pyodbc.connect(DataSource)
    cursor = conexionDB.cursor()

    for user in Usuarios:

        # Se determina el ID del usuario propietario de las tablas
        sql_creador  = "SELECT * FROM SYSUSERS WHERE NAME = " + "'" + user + "'"
        cursor.execute(sql_creador)
        row = cursor.fetchone()
        creator_id=row.suid

        # Se extrae la lista de tablas de las cuales el usuario es propitario 
        sql_tablas = "SELECT * FROM SYSTABLE WHERE TABLE_TYPE = \'BASE\'"
        sql_tablas = sql_tablas + " AND CREATOR=" + str(creator_id)
        sql_tablas = sql_tablas + " ORDER BY TABLE_NAME "
        cursor.execute(sql_tablas)
        rows_table = cursor.fetchall()

        modeloDatos[user] = {}
        
        for row_table in rows_table:
    
            modeloDatos[user][row_table.table_name] = {
            "table_id" : row_table.table_id,
            "table_count" : row_table.count }

            #print(row_table.table_name)

            # Extrae las columnas de cada tabla
            sql_columnas  = "SELECT * FROM SYSCOLUMNS WHERE TNAME = " + "'" + row_table.table_name + "'" + " ORDER BY COLNO"
            
            cursor.execute(sql_columnas)
            rows_columns = cursor.fetchall()
            modeloDatos[user][row_table.table_name]["table_columns"] = []

            for row_column in rows_columns:
                obj = 
                obj["col_name"]=row_column.cname
                obj["col_type"]=row_column.coltype
                obj["col_null"]=row_column.nulls
                obj["col_length"]=row_column.length
                obj["col_syslength"]=row_column.syslength
                obj["col_inPK"]=row_column.in_primary_key
                modeloDatos[user][row_table.table_name]["table_columns"].append(obj)

                print("\t",row_column.cname)

            modeloDatos[user][row_table.table_name]["table_indexes"] = []

            sql_indices  = "SELECT * FROM SYSINDEXES WHERE INDEXTYPE = 'Non-unique' AND TNAME =" + "'" + row_table.table_name + "' order by iname"   

            cursor.execute(sql_indices)
            rows_indices = cursor.fetchall()
            
            for rows_indice in rows_indices:
                obj={}
                obj["ind_name"]=rows_indice.iname
                colnames=rows_indice.colnames
                obj["columns"]=colnames.split(',')
                modeloDatos[user][row_table.table_name]["table_indexes"] .append(obj)

                print("\t",rows_indice.tname)
        

    cursor.close()
    conexionDB.close()

    return modeloDatos

In [58]:
modeloDatos = crearModeloDatos(DataSource,Usuarios)

ABC
ABCObsoletos
ABCUnidades
AClientes
AContar
AFPs
AlarmaCobros
Anulaciones
Apartados
Aplicaciones
AuditoriaClientes
AuditoriaContratosD
AuditoriaCreditos
AuditoriaDescuentos
AuditoriaInventario
AuditoriaPrecios
AuditoriaProductos
AuditoriaReimpresiones
AuditoriaTraslados
AuditoriaUbicaciones
AutCredito
Autorizaciones
Bancos
BCodigos
BComprobantes
BCuentas
BCuentasD
BCuentasM
BFormatos
BMovimientos
BonificacionesDescuentos
Bultos
CambioPreciosD
CambioPreciosM
CancelacionContratos
CanceladasD
CanceladasM
Caracteristicas
CategoriaCaracteristicas
Categorias
CAutorizaciones
CCFacturaCabeza
CCFacturaCuerpo
CCierresD
CCierresM
CCInvData0008
CCInvData0009
CClasificaciones
CCortePartidas
CCortesM
CCuentas
CCuentaSaldos
CCuentasD
CCuentasDepreciacion
CCuentasSaldo
CDiresa
CIERRECXCD
CIERRECXCM
ClasesCliente
ClasesDocumento
ClasesDocumentoBancario
ClasesDocumentoTiposGestor
ClasesGestor
ClasesProveedor
ClasificacionMora
ClavesCalculadas
ClienteClases
ClienteCupones
ClienteDescuentos
ClienteDirD

In [59]:
file = open("ModeloDatos.txt", "w" )
file.write(str(modeloDatos))
file.close()

In [61]:
import json
file = open("ModeloDatos.json", "w" )
file.write(json.dumps(modeloDatos))
file.close()